<a href="https://colab.research.google.com/github/TimWingers/MyPythonScripts/blob/master/imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Установим последнюю версию TensorFlow
!pip install tensorflow==2.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 45.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.1
    Uninstalling tensorboard-data-server-0.7.1:
      Successfully uninstalled tensorboard-data-server-0.7.1
  Attempting uninstall: google-auth-oauthlib
    Found existing insta

In [2]:
#Импортируем необходимые библиотеки
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

In [3]:
#Загрузим датасет с отзывами Imdb.

#Импортируем модуль pathlib для удаленной загрузки файлов.
from pathlib import Path

#Укажем url адрес где лежат данные и имя файла
DOWNLOAD_ROOT = "http://ai.stanford.edu/~amaas/data/sentiment/"
FILENAME = "aclImdb_v1.tar.gz"

#С помощью библиотеки Keras загрузим файлы с автоматической распаковкой
filepath = keras.utils.get_file(FILENAME, DOWNLOAD_ROOT + FILENAME, extract=True)

#Загрузим в переменную "path" путь к родительской директории файлов,
# а также добавим "Imdb" в конец пути
path = Path(filepath).parent / "aclImdb"
path

84140032/84125825 [==============================] - 10s 0us/step


PosixPath('/root/.keras/datasets/aclImdb')

In [4]:
#Ок. Теперь все файлы с данными у нас лежат в папке "aclImdb".
#Давайте посмотрим на их структуру.

#Запустим цикл, который будет проходить по всем файлам и папкам в указанном path (пути).
#name будет содержать текущее имя папки или файла, subdirs будет содержать список подпапок
# в текущей папке, а files будет содержать список файлов в текущей папке.
for name, subdirs, files in os.walk(path):
    indent = len(Path(name).parts) - len(path.parts) #эта строчка считает отступ в зависимости от того, насколько глубоко находится текущая папка
    print("    " * indent + Path(name).parts[-1] + os.sep) #имя текущей папки с отступом, используя символы пробела, os.sep используем для добавления правильного разделителя пути
    #Запускаем вложенный цикл для перебора файлов в текущей папке. index будет содержать текущий индекс файла, а filename - имя из списка files.
    for index, filename in enumerate(sorted(files)):
        if index == 3: #если текущий индекс равен 3
            print("    " * (indent + 1) + "...") #выводим троеточие с отступом, увеличенным на один
            break
        print("    " * (indent + 1) + filename)  #если не равен 3, то выводим имя файла с отступом +1

aclImdb/
    README
    imdb.vocab
    imdbEr.txt
    train/
        labeledBow.feat
        unsupBow.feat
        urls_neg.txt
        ...
        neg/
            0_3.txt
            10000_4.txt
            10001_4.txt
            ...
        pos/
            0_9.txt
            10000_8.txt
            10001_10.txt
            ...
        unsup/
            0_0.txt
            10000_0.txt
            10001_0.txt
            ...
    test/
        labeledBow.feat
        urls_neg.txt
        urls_pos.txt
        neg/
            0_2.txt
            10000_4.txt
            10001_1.txt
            ...
        pos/
            0_10.txt
            10000_7.txt
            10001_9.txt
            ...


In [5]:
#Давайте теперь посмотрим сколько данных у нас содержится в обучаемых
# и тестовых выборках с положительными и негативными отзывами.

#Создадим функцию с аргументом dirpath. Она будет принимать путь к директории и
# возврать список путей ко всем файлам с расширением ".txt"
def review_paths(dirpath):
    return [str(path) for path in dirpath.glob("*.txt")]

train_pos = review_paths(path / "train" / "pos")
train_neg = review_paths(path / "train" / "neg")
test_valid_pos = review_paths(path / "test" / "pos")
test_valid_neg = review_paths(path / "test" / "neg")

len(train_pos), len(train_neg), len(test_valid_pos), len(test_valid_neg)

(12500, 12500, 12500, 12500)

In [6]:
#Поскольку датасет довольно большой (по 20Мб для train и test), то нам придется
# урезать часть данных, чтобы они влезли в ОЗУ.

#Для этого мы перемешаем элементы массива в случайном порядке и отберем 10000 из них.
np.random.shuffle(test_valid_pos)

test_pos = test_valid_pos[:10000]
test_neg = test_valid_neg[:10000]
valid_pos = test_valid_pos[10000:]
valid_neg = test_valid_neg[10000:]

In [7]:
#Теперь нам нужно привести данные в формат цельного датасета.

#Напишем функцию, которая будет принимать пути с положительными и негативными
# отзывами, создаст списки с отзывами и метками классов
def imdb_dataset(filepaths_positive, filepaths_negative):
    reviews = []
    labels = []
    #Создаем цикл для перебора всех путей и меток в filepaths_negative и filepaths_positive
    for filepaths, label in ((filepaths_negative, 0), (filepaths_positive, 1)):
        for filepath in filepaths:
            #Открываем каждый файл и добавляем в список
            with open(filepath) as review_file:
                reviews.append(review_file.read())
            #Открываем каждую метку и добавляем в список
            labels.append(label)
    #Возвращаем датасет, содержащий 2 тензора: tf.constant(reviews) и tf.constant(labels).
    return tf.data.Dataset.from_tensor_slices((tf.constant(reviews), tf.constant(labels)))

In [8]:
#Теперь проверим все ли импортировалось корректно.
#Возьмем 3 случайных отзыва и метки из созданного датасета.
for X, y in imdb_dataset(train_pos, train_neg).take(3):
    print(X)
    print(y)
    print()

tf.Tensor(b"This was really a very bad movie. I am a huge fan of Italian Horror, Argento, Mario Bava, Fulci and yes, even our good friend here Lamberto sometimes comes out with a good one. I found the first two 'Demons' films to be highly entertaining - they were so bad they were great but this one is just so bad that it is really, really bad. It is intensely boring, the story never goes anywhere and I hated the characters - the wife slapping husband and whiny cry-baby pain in the *** wife drove me mad, there was nowhere near enough of the story devoted to the Ogre who was probably the best actor in the whole film. I turned it off about three quarters of the way through because I was very, very BORED! Don't bother.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int32)

tf.Tensor(b'I voted 3 for this movie because it looks great as does all of Greenaways output. However it was his usual mix of "art" sex and pretentious crap.I know lots of people like this film but I grew tired o

In [9]:
#Все корректно. 1й тензор у нас содержит текст, помеченный меткой b, что означает его начало.
#2й тензор у нас содержит метку (оценку) этого комментариях в формате целого числа.

#Теперь можно приступать к препроцессингу данных для загрузки в нейросеть.

#Определим размер батча.
batch_size = 32

#Создадим обучаемый, тестовый и валидационные наборы данных.
#Будем использовать shuffle для обучения, для перетасовки его элементов случайным образом.
#А также будем использовать prefetch для автопредзагрузки 1 пакета, чтобы ускорить обучение.
train_set = imdb_dataset(train_pos, train_neg).shuffle(50000).batch(batch_size).prefetch(1)
valid_set = imdb_dataset(valid_pos, valid_neg).batch(batch_size).prefetch(1)
test_set = imdb_dataset(test_pos, test_neg).batch(batch_size).prefetch(1)

In [10]:
#Теперь нам необходимо профильтровать весь "мусор" в отзывах.

#Напишем функцию, которая будет принимать батчами по 50 слов наши данные и
# преобразовывать для обучения нейросети.
def preprocess(X_batch, n_words=50):
    #Выравниваем границы матриц с текстами, по размеру батча, умножая на [1, 0],
    # и добавляем к ним [0, n_words], чтобы увеличить количество столбцов на n_words
    shape = tf.shape(X_batch) * tf.constant([1, 0]) + tf.constant([0, n_words])
    #берем первые 500 символов из каждого текста в пакете X_batch и сохраняем в Z
    Z = tf.strings.substr(X_batch, 0, 500)
    #преобразуем все тексты в нижний регистр
    Z = tf.strings.lower(Z)
    #удаляем все теги "<br>" и "<br/> из текста
    Z = tf.strings.regex_replace(Z, b"<br\\s*/?>", b" ")
    #заменяем все небуквенные символы (не входящие в a до z) на пробелы
    Z = tf.strings.regex_replace(Z, b"[^a-z]", b" ")
    #разделяем все предложения на слова и сохраняем в списки слов
    Z = tf.strings.split(Z)
    #возвращаем данные в виде тензора размером shape, заполняя пустоты pad'ами.
    return Z.to_tensor(shape=shape, default_value=b"<pad>")

In [11]:
#Давайте проверим все ли преобразовалось корректно
X_example = tf.constant(["It's a great, great movie! I loved it.", "It was terrible, run away!!!"])
preprocess(X_example)

<tf.Tensor: shape=(2, 50), dtype=string, numpy=
array([[b'it', b's', b'a', b'great', b'great', b'movie', b'i', b'loved',
        b'it', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>'],
       [b'it', b'was', b'terrible', b'run', b'away', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'

In [12]:
#Как мы видим все корректно. Однако тензор очень разрешен, в нем множество pad-ов.
#Для наглядности мы можем отсеить все pad-ы и посмотреть что же там внутри.

#Импортируем библиотеку счетчика
from collections import Counter

#И напишем функцию которая отсортирует все значения в тензоре по убыванию
# и выведем все уникальные на экран.
def get_vocabulary(data_sample, max_size=1000):
    preprocessed_reviews = preprocess(data_sample).numpy()
    counter = Counter()
    for words in preprocessed_reviews:
        for word in words:
            if word != b"<pad>":
                counter[word] += 1
    return [b"<pad>"] + [word for word, count in counter.most_common(max_size)]

get_vocabulary(X_example)

[b'<pad>',
 b'it',
 b'great',
 b's',
 b'a',
 b'movie',
 b'i',
 b'loved',
 b'was',
 b'terrible',
 b'run',
 b'away']

In [13]:
#Как и ожидалось, pad-ы доминируют. Также как и предлоги и союзы, хотя в этом отзыве
# автор также явно находится под впечатлением используя часто 'great', 'movie' и 'loved'.

In [14]:
#Итак препроцессинг почти завершен. Однако компьютеры не умеют работать со словами, а
# только с цифрами. Поэтому нам необходимо преобразовать каждый токен наших тензоров
# в числа.

#Будем использовать библиотеку TextVectorization
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
text_vectorization = TextVectorization()

#Будем использовать метод adapt(), который автоматически адаптируется к данным и
# позволяет выявить наиболее часто встречающиеся слова и их векторное представление.
text_vectorization.adapt(X_example)
text_vectorization(X_example)

<tf.Tensor: shape=(2, 8), dtype=int64, numpy=
array([[ 9, 12,  3,  3,  7, 10,  8,  2],
       [ 2,  4,  5,  6, 11,  0,  0,  0]])>

In [15]:
#Теперь наш текст, который мы видели выше, представляет собой массив с числами.
#В таком формате мы и будем передавать наши данные в нейросеть.

In [16]:
#Давайте теперь приведем не только этот текст, но и все тексты нашего датасета
# в такой формат.

#Определим максимальный размер словаря для преобразования (то есть количество
# уникальных слов, которое мы хотим учесть при обучении модели)
max_vocabulary_size = 3000
#Определим количество "корзин" (buckets), которые будут использоваться для
# представления неизвестных слов (OOV - Out of Vocabulary). Благодаря этому методу
# сеть сможет обрабатывать слова, которые не были включены в словарь.
n_oov_buckets = 100

#Теперь создадим переменную которая содержит только тексты из обучающего набора данных.
# Используем метод map, чтобы извлечь только тексты и игнорировать метки (label) отзывов.
sample_review_batches = train_set.map(lambda review, label: review)

#Объединим все тексты в 1 большой массив sample_reviews.
sample_reviews = np.concatenate(list(sample_review_batches.as_numpy_iterator()),
                                axis=0)

#И запустим процесс векторизации, где output_sequence_length - длина выходной последовательности
# которую мы получим после векторизации текста.
text_vectorization = TextVectorization(max_tokens=max_vocabulary_size,
                                       standardize='lower_and_strip_punctuation',
                                       output_sequence_length=n_oov_buckets,
                                       input_shape=[])
text_vectorization.adapt(sample_reviews)

In [17]:
#Давайте теперь посмотрим что получилось
text_vectorization(X_example)

<tf.Tensor: shape=(2, 100), dtype=int64, numpy=
array([[ 30,   4,  85,  85,  18,  10, 434,   9,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  9,  14, 384, 530, 242,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,

In [18]:
#Мы получили весьма разреженный тензор размером 2х100, где числа представляют
# собой нечто иное как кодировку слов в отзыве.

In [19]:
#Теперь нам необходимо преобразовать данные в мешок слов (bag of words).

#Создадим соответствующий класс
class BagOfWords(keras.layers.Layer):
    #Объявим параметры (число токенов, тип переменных, именнованные аргументы)
    def __init__(self, n_tokens, dtype=tf.int32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        self.n_tokens = n_tokens
    #Создадим функцию для приема input-ов и one-hot энкодинга
    def call(self, inputs):
        #преобразуем значения в one-hot кодировку, где каждое значение представляется
        # в виде вектора с n_tokens элементами, и элемент с индексом inputs[i] равен 1,
        # а все остальные элементы равны 0. Это позволит представить каждый токен в виде
        # уникального вектора.
        one_hot = tf.one_hot(inputs, self.n_tokens)
        #вычислим сумму one_hot представлений по оси 1 (вертикальной), используя функцию
        # tf.reduce_sum, которая складывает все элементы вдоль указанной оси. Результатом
        # операции будет вектор сумм всех элементов каждого one_hot представления.
        return tf.reduce_sum(one_hot, axis=1)[:, 1:]

In [20]:
#Проверим работу класса и его функций.
bag_of_words = BagOfWords(n_tokens=4)
simple_example = tf.constant([[1, 3, 1, 0, 0], [2, 2, 0, 0, 0]])
bag_of_words(simple_example)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[2., 0., 1.],
       [0., 2., 0.]], dtype=float32)>

In [21]:
#Создадим переменную n_tokens, которая равна сумме трех других переменных:
# max_vocabulary_size, n_oov_buckets и 1. Единица - это наши паддинги, для выравнивания
# shape тензоров, max_vocabulary_size - это число, которое представляет размер словаря,
# то есть количество уникальных слов, которые будут использоваться, а n_oov_buckets - это
# количество "темных ячеек вне словаря" - они позволят модели обрабатывать слова, которых
# нет в словаре.
n_tokens = max_vocabulary_size + n_oov_buckets + 1

#И положим все эти токены в наш мешок слов.
bag_of_words = BagOfWords(n_tokens)

In [22]:
#Ок. Теперь пора переходить к обучению нейросети.

#Для начала укажем параметры будущей сетки.
model = keras.models.Sequential([  #секвентальная модель от Keras
    text_vectorization,
    bag_of_words,
    keras.layers.Dense(200, activation="relu", #1й слой - 200 нейронов, функция активации ReLu
                       kernel_regularizer=keras.regularizers.l2(0.001)), #регуляризация L2 (для избежания переобучения)
    keras.layers.Dropout(0.2),                 #слой дропаута для лучшей сходимости
    keras.layers.Dense(100, activation="relu", #2й слой - 100 нейронов, активация через ReLu
                       kernel_regularizer=keras.regularizers.l2(0.001)), #также регуляризация через L2
    keras.layers.Dense(1, activation="sigmoid")  #финишный слой с 1 нейроном, активация функцией сигмоиды
])

#Собираем модель. В качестве функции потерь будем использовать бинарную кросс-энтропию,
# оптимизатор - "nadam", метрика качества - "accuracy".
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])

#Добавим функцию ранней остановки. В случае если модель не улучшается 3 эпохи
# подряд - то остановить обучение.
early_stopping = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

In [23]:
#Запускаем обучение модели. Будем использовать 12 эпох
model.fit(train_set, epochs=12, validation_data=valid_set, callbacks=[early_stopping])

Epoch 1/12
782/782 [==============================] - 55s 69ms/step - loss: 0.6300 - accuracy: 0.7771 - val_loss: 0.5521 - val_accuracy: 0.7946
Epoch 2/12
782/782 [==============================] - 53s 68ms/step - loss: 0.5201 - accuracy: 0.8149 - val_loss: 0.5672 - val_accuracy: 0.7794
Epoch 3/12
782/782 [==============================] - 49s 62ms/step - loss: 0.4924 - accuracy: 0.8318 - val_loss: 0.5282 - val_accuracy: 0.7986
Epoch 4/12
782/782 [==============================] - 49s 62ms/step - loss: 0.4733 - accuracy: 0.8452 - val_loss: 0.5332 - val_accuracy: 0.8052
Epoch 5/12
782/782 [==============================] - 49s 62ms/step - loss: 0.4528 - accuracy: 0.8606 - val_loss: 0.5577 - val_accuracy: 0.8006
Epoch 6/12
782/782 [==============================] - 49s 62ms/step - loss: 0.4348 - accuracy: 0.8751 - val_loss: 0.5989 - val_accuracy: 0.7936


In [24]:
#Итак, обучение было остановлено на 5й эпохе, потому что последующие не приводили
# к улучшениям. Тем не менее мы получили модель, которая весьма хорошо предсказывает
# оценки по отзывам в imdb: 86% на обучаемой выборке и 80.5% - на тестовой.

In [57]:
#Теперь осталось проверить нашу модель в деле.
#Создадим интерактивную оболочку с окном где можно писать отзыв о фильме,
# а наша модель будет автоматически выставлять оценки.
import ipywidgets as widgets
from IPython.display import display

#Создаем функцию отображения окна и кнопки
def predict_review_rating(model):
    review_text = widgets.Textarea(description='Отзыв о фильме:', rows=5)
    button = widgets.Button(description='Предсказать рейтинг')
    output = widgets.Output()

    def on_button_clicked(b):
        with output:
            rating = model.predict([review_text.value])[0]
            print('Прогнозный рейтинг:', rating*10) #в нейросетке шкала от 0 до 1
            if rating >= 0.7:
              print('Положительный отзыв')
            elif 0.45 <= rating < 0.7:
              print('Нейтральный отзыв')
            else:
              print('Отрицательный отзыв')

    #вывод результатов на экран
    button.on_click(on_button_clicked)
    display(review_text, button, output)

In [58]:
predict_review_rating(model)

Textarea(value='', description='Отзыв о фильме:', rows=5)

Button(description='Предсказать рейтинг', style=ButtonStyle())

Output()

In [59]:
predict_review_rating(model)

Textarea(value='', description='Отзыв о фильме:', rows=5)

Button(description='Предсказать рейтинг', style=ButtonStyle())

Output()

In [60]:
predict_review_rating(model)

Textarea(value='', description='Отзыв о фильме:', rows=5)

Button(description='Предсказать рейтинг', style=ButtonStyle())

Output()

In [66]:
print()